In [2]:
from pyspark.sql import SparkSession

In [3]:
# spark = SparkSession.builder.appName('weather').getOrCreate()

In [4]:
# df = spark.read.csv("C:/Users/SZ/PycharmProjects/berlin_weather/ber_weather_combined_1876_2019.csv", inferSchema=True, header=True)

In [5]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-e0d42c62-511a-4705-9fd5-f4e9b8c462d7',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'M8YydFolrb_17ua7e3k8XYOKspbt0TlNcrfaxdyCUp4M'
}

configuration_name = 'os_ef22993239a040aba5e03c8fb92d4476_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('ber_weather_combined_1876_2019.csv', 'tryspark-donotdelete-pr-qgg5ygyxi1owvr'))
df_data_1.take(5)


[Row(Measurement day='1876-01-01', Max daily temp='3.5', Min daily temp='-3.0', Medium daily temp='2.2'),
 Row(Measurement day='1876-01-02', Max daily temp='2.8', Min daily temp='2.3', Medium daily temp='2.5'),
 Row(Measurement day='1876-01-03', Max daily temp='2.8', Min daily temp='-1.8', Medium daily temp='0.3'),
 Row(Measurement day='1876-01-04', Max daily temp='-1.8', Min daily temp='-7.0', Medium daily temp='-5.8'),
 Row(Measurement day='1876-01-05', Max daily temp='-6.8', Min daily temp='-13.3', Medium daily temp='-9.8')]

In [6]:
df = df_data_1

In [7]:
df.columns

['Measurement day', 'Max daily temp', 'Min daily temp', 'Medium daily temp']

In [8]:
df.head()

Row(Measurement day='1876-01-01', Max daily temp='3.5', Min daily temp='-3.0', Medium daily temp='2.2')

In [9]:
df.printSchema()

root
 |-- Measurement day: string (nullable = true)
 |-- Max daily temp: string (nullable = true)
 |-- Min daily temp: string (nullable = true)
 |-- Medium daily temp: string (nullable = true)



In [10]:
df.describe().show()

+-------+---------------+------------------+-----------------+-----------------+
|summary|Measurement day|    Max daily temp|   Min daily temp|Medium daily temp|
+-------+---------------+------------------+-----------------+-----------------+
|  count|          52400|             52400|            52400|            52400|
|   mean|           null|13.218114503816638|5.543538167938865|  9.3734522900764|
| stddev|           null| 9.045948710369242|7.062744766260845|7.882679311753339|
|    min|     1876-01-01|              -0.1|             -0.1|             -0.1|
|    max|     2019-12-31|               9.9|              9.9|              9.9|
+-------+---------------+------------------+-----------------+-----------------+



In [11]:
df_new_1 = df.withColumn('New_col', df['Max daily temp'] - df['Min daily temp']).select(['New_col'])

In [12]:
df_new_1.show()

+------------------+
|           New_col|
+------------------+
|               6.5|
|               0.5|
|               4.6|
|               5.2|
| 6.500000000000001|
| 7.800000000000001|
|               2.8|
| 3.500000000000001|
|               4.0|
|7.1000000000000005|
|               2.2|
|               2.8|
|               1.5|
|               2.5|
|1.2999999999999998|
|               3.0|
|               3.5|
|               4.8|
|               3.0|
|               4.0|
+------------------+
only showing top 20 rows



In [13]:
df.orderBy(df['Max daily temp'].desc()).select(['Measurement day']).head(1)[0]['Measurement day']

'1882-12-28'

In [14]:
from pyspark.sql.functions import mean

In [15]:
df.select(mean('Max daily temp')).show()

+-------------------+
|avg(Max daily temp)|
+-------------------+
| 13.218114503816638|
+-------------------+



In [21]:
from pyspark.sql.functions import max, min

df.select(max('Max daily temp'), min('Max daily temp')).show()

+-------------------+-------------------+
|max(Max daily temp)|min(Max daily temp)|
+-------------------+-------------------+
|                9.9|               -0.1|
+-------------------+-------------------+



In [25]:
df.filter(df['Max daily temp'] > 30).count()

698

In [26]:
df.filter(df['Max daily temp'] < -5).count()

517

In [29]:
df_max = df.filter(df['Max daily temp'] > 35)
df_max.show()

+---------------+--------------+--------------+-----------------+
|Measurement day|Max daily temp|Min daily temp|Medium daily temp|
+---------------+--------------+--------------+-----------------+
|     1904-07-16|          36.4|          20.7|             28.4|
|     1905-07-01|          36.3|          21.8|             28.5|
|     1943-08-03|          36.6|          20.4|             27.6|
|     1943-08-20|          36.0|          17.0|             26.5|
|     1943-08-21|          36.6|          19.4|             26.9|
|     1949-08-08|          36.4|          17.6|             24.6|
|     1959-07-10|          37.0|          20.4|             28.4|
|     1959-07-11|          38.1|          19.7|             29.6|
|     1983-08-01|          36.1|          19.0|             25.2|
|     1984-07-11|          36.4|          21.0|             30.0|
|     1992-08-09|          37.1|          19.0|             29.8|
|     1994-07-30|          36.2|          20.5|             29.1|
|     1994

In [31]:
from pyspark.sql.functions import (dayofmonth, hour, dayofyear, month, year, weekofyear, format_number, date_format)

year_df = df.withColumn('Year', year(df['Measurement day']))

year_df.groupBy('Year').max()['Year', 'max(Max daily temp)'].show()

AnalysisException: "cannot resolve '`max(Max daily temp)`' given input columns: [Year, max(Year)];;\n'Project [Year#382, 'max(Max daily temp)]\n+- Aggregate [Year#382], [Year#382, max(Year#382) AS max(Year)#394]\n   +- Project [Measurement day#10, Max daily temp#11, Min daily temp#12, Medium daily temp#13, year(cast(Measurement day#10 as date)) AS Year#382]\n      +- Relation[Measurement day#10,Max daily temp#11,Min daily temp#12,Medium daily temp#13] csv\n"